In [17]:
import numpy as np

irradiance385 = 8.8  # mW/cm²
irradiance405 = 8  # mW/cm²

times = np.array([0.8, 1, 1.2, 1.4, 1.7, 2.1, 2.5, 3, 3.6, 4.4, 5.2, 6.3, 7.6, 9.1, 10.9, 13.1]) #seconds

# Calculate radiant exposure in mJ/cm²:
radiant_exposure385 = irradiance385 * times
radiant_exposure405 = irradiance405 * times

print("Exposure time (s) | 405nm Radiant Exposure (mJ/cm²) | 385nm Radiant Exposure (mJ/cm²)")
for t, H1, H2 in zip(times, radiant_exposure405, radiant_exposure385):
    print(f"{t:>15} | {H1:>22.5f} | {H2:>22.5f}")


Exposure time (s) | 405nm Radiant Exposure (mJ/cm²) | 385nm Radiant Exposure (mJ/cm²)
            0.8 |                6.40000 |                7.04000
            1.0 |                8.00000 |                8.80000
            1.2 |                9.60000 |               10.56000
            1.4 |               11.20000 |               12.32000
            1.7 |               13.60000 |               14.96000
            2.1 |               16.80000 |               18.48000
            2.5 |               20.00000 |               22.00000
            3.0 |               24.00000 |               26.40000
            3.6 |               28.80000 |               31.68000
            4.4 |               35.20000 |               38.72000
            5.2 |               41.60000 |               45.76000
            6.3 |               50.40000 |               55.44000
            7.6 |               60.80000 |               66.88000
            9.1 |               72.80000 |              

In [3]:
import numpy as np
a = np.zeros([3,3])

a.shape

(3, 3)